# General Imports


In [ ]:
import os
import inspect
import sys
import pandas as pd
import charts
from opengrid import config
c = config.Config()

from opengrid.library import misc, houseprint

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

## Houseprint

In [ ]:
hp = houseprint.Houseprint()
hp.init_tmpo()
hp._tmpos.debug = False
hp.sync_tmpos()

## Create dataframes with minute data for a single year, by sensortype

In [ ]:
for sensortype in ['water', 'gas', 'electricity']:
    print('Processing {}'.format(sensortype))
    for y in ['2015', '2016', '2017']:
        print('year {}'.format(y))
        head = pd.Timestamp('{}0101'.format(y), tz='Europe/Brussels')
        tail = pd.Timestamp('{}0101'.format(int(y)+1), tz='Europe/Brussels')
        df = hp.get_data(sensortype=sensortype, head=head, tail=tail, diff=True, resample='min')
        df.rename(columns=lambda x: x[:4], inplace=True)
        path = os.path.join(c.get('data', 'folder'), '{}_{}_min.pickle'.format(sensortype, y))
        df.to_pickle(path)

## Create dataframes with hourly, daily and monthly data

In [ ]:
for sensortype in ['water', 'gas', 'electricity']:
    print('Processing {}'.format(sensortype))
    for y in ['2015', '2016', '2017']:
        print('year {}'.format(y))
        path_min = os.path.join(c.get('data', 'folder'), '{}_{}_min.pickle'.format(sensortype, y))
        df = pd.read_pickle(path_min)
        # hourly: mean values
        df_hour = df.resample(rule='H').mean()
        path_hour = os.path.join(c.get('data', 'folder'), '{}_{}_hour.pickle'.format(sensortype, y))
        df_hour.to_pickle(path_hour)
        # daily: totals in  liter/day, kWh/day or kWh/day
        #df_day = df_hour.resample(rule='D').mean()
        #path_day = os.path.join(c.get('data', 'folder'), '{}_{}_day.pickle'.format(sensortype, y))
        #df_day.to_pickle(path_day)

In [ ]:
misc.unit_conversion_factor(source='liter/min', target='m**3/hour')